In [27]:
from moffragmentor import MOF
from pymatgen.core import Structure 


In [12]:
m = MOF.from_cif('ABEDAQ.cif')

/Users/kevinmaikjablonka/miniconda3/envs/moffragmentor/lib/python3.8/site-packages/pymatgen/io/cif.py:709: UserWarning: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
  warnings.warn(msg)
/Users/kevinmaikjablonka/miniconda3/envs/moffragmentor/lib/python3.8/site-packages/pymatgen/io/cif.py:1164: UserWarning: Issues encountered while parsing CIF: No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [13]:
f = m.fragment()

In [14]:
def count_radical_electrons(mol):
    radical_electrons = []
    for atom in mol.GetAtoms():
        radical_electrons.append(atom.GetNumRadicalElectrons())
    return sum(radical_electrons)

In [17]:
def get_linker_charges(fragments): 
    linker_charges = []
    
    for linker in fragments.linkers: 
        linker_charges.append(count_radical_electrons(linker.rdkit_mol))
    
    return linker_charges, sum(linker_charges)

In [18]:
_, lc = get_linker_charges(f)
print(lc)

24


In [21]:
def get_bound_solvent_charges(fragments): 
    mol_charges = []
    
    for solvent in fragments.unbound_solvent: 
        mol_charges.append(count_radical_electrons(solvent.rdkit_mol))
    
    return mol_charges, sum(mol_charges)

In [22]:
_, bsc = get_bound_solvent_charges(f)
print(bsc)

0


In [23]:
CHARGE_RULES = {
        'O': -2,
        'H' : 1, 
        'F': -1, 
        'Cl': -1, 
        'Br': -1,
        'I': -1
}

In [24]:
def count_charge_in_composition(composition_dict): 
    charge = 0
    for k, v in composition_dict.items():
        charge += CHARGE_RULES[k] * v
        
    return charge

In [25]:
def select_sites(mof, selected_indices): 
    sites = []
    
    for i in selected_indices: 
        sites.append(mof.structure[i])
        
    s = Structure.from_sites(sites)
    
    return s

In [28]:
def get_node_charges(mof, fragments):
    
    node_charges = []
    
    for n in fragments.nodes:
        indices = set(n._original_indices) - n._original_connecting_paths - set(fragments.bound_solvent.indices) - set(mof.metal_indices)
        
        s = select_sites(mof, indices)
        
        charge = count_charge_in_composition(s.composition.as_dict())
        
        node_charges.append(charge) 
    
    return sum(node_charges) 

In [29]:
get_node_charges(m, f)

-4.0

In [ ]:
def get_oxidation_states(mof): 
    ...